### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = tomorrow

In [5]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5193,2023-10-22,Austrália Nbl,00:00,Melbourne United,NZ Breakers,1.48,2.65,167.5,1.97,1.85,-4.5,1.91,1.91
5194,2023-10-22,Japão B.League,01:35,Shimane,Saga,1.15,4.75,153.5,1.83,1.83,-10.5,1.83,1.83
5195,2023-10-22,Austrália Nbl,02:00,Sydney Kings,Tasmania JackJumpers,1.50,2.60,174.5,1.91,1.91,-4.5,1.91,1.91
5196,2023-10-22,Coreia Do Sul Kbl,02:00,Seoul Knights,Suwon KT,1.37,3.20,163.5,1.91,1.91,-6.5,1.91,1.91
5197,2023-10-22,Coreia Do Sul Kbl,02:00,Goyang Sono,Wonju DB,1.91,1.91,168.5,1.91,1.91,-1.5,1.91,1.91
5198,2023-10-22,Japão B.League,02:05,Shinshu,Diamond Dolphins,4.00,1.22,162.5,1.83,1.83,6.5,1.87,1.80
5199,2023-10-22,Japão B.League,02:05,Yokohama,Ryukyu,2.25,1.57,160.5,1.83,1.83,1.5,1.83,1.83
5200,2023-10-22,Japão B.League,02:05,Hiroshima D.,Utsunomiya Brex,2.20,1.63,149.5,1.83,1.83,2.5,1.83,1.83
5201,2023-10-22,Japão B.League,02:05,Akita,Kyoto,1.56,2.30,155.5,1.83,1.83,-5.5,1.83,1.83
5202,2023-10-22,Japão B.League,03:05,SeaHorses Mikawa,Hokkaido,1.29,3.50,159.5,1.83,1.83,-7.5,1.83,1.83


## Apostas do dia

### Over v1 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Austrália Nbl,Melbourne United,NZ Breakers,167.5,1.97,0.7221,Over
2,02:00,Austrália Nbl,Sydney Kings,Tasmania JackJumpers,174.5,1.91,0.8381,Over
3,02:00,Coreia Do Sul Kbl,Seoul Knights,Suwon KT,163.5,1.91,0.8381,Over
4,02:00,Coreia Do Sul Kbl,Goyang Sono,Wonju DB,168.5,1.91,0.8454,Over
13,04:00,Coreia Do Sul Kbl,KCC Egis,Seoul Thunders,163.5,1.95,0.7853,Over
15,08:00,Espanha Acb,Tenerife,Real Madrid,166.5,1.91,0.7189,Over
21,09:30,França Lnb,Lyon-Villeurbanne,Roanne,164.5,1.91,0.8499,Over
23,10:30,Alemanha Bbl,Syntainics MBC,Bamberg,172.5,1.91,0.8244,Over
24,10:30,Alemanha Bbl,Chemnitz,Gottingen,173.5,1.91,0.8302,Over
25,11:15,Grécia Liga De Basquete,Kolossos Rhodes,Olympiakos,156.5,1.91,0.8011,Over


### Over v1 (LDA)

In [10]:
# Duplicando o dataset
df_over_v1 = jogos.copy()

df_over_v1.reset_index(drop=True, inplace=True)
df_over_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v1.empty:
    mdl_over_v1 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v1, df_over_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,02:00,Coreia Do Sul Kbl,Seoul Knights,Suwon KT,163.5,1.91,0.8381,Over
13,04:00,Coreia Do Sul Kbl,KCC Egis,Seoul Thunders,163.5,1.95,0.7853,Over
15,08:00,Espanha Acb,Tenerife,Real Madrid,166.5,1.91,0.7189,Over
21,09:30,França Lnb,Lyon-Villeurbanne,Roanne,164.5,1.91,0.8499,Over
24,10:30,Alemanha Bbl,Chemnitz,Gottingen,173.5,1.91,0.8302,Over
25,11:15,Grécia Liga De Basquete,Kolossos Rhodes,Olympiakos,156.5,1.91,0.8011,Over
26,11:30,França Lnb,Strasbourg,Monaco,163.5,1.91,0.8389,Over
28,12:00,Alemanha Bbl,Alba Berlin,Vechta,174.5,1.91,0.8304,Over
29,12:00,Espanha Acb,Valencia,Manresa,168.5,1.91,0.8339,Over
30,12:00,Itália Liga A,Treviso,Venezia,160.5,1.91,0.8303,Over


### Over v2 (LDA)

In [11]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,02:00,Coreia Do Sul Kbl,Seoul Knights,Suwon KT,163.5,1.91,0.6398,Over
13,04:00,Coreia Do Sul Kbl,KCC Egis,Seoul Thunders,163.5,1.95,0.9003,Over
15,08:00,Espanha Acb,Tenerife,Real Madrid,166.5,1.91,0.5932,Over
21,09:30,França Lnb,Lyon-Villeurbanne,Roanne,164.5,1.91,0.6305,Over
24,10:30,Alemanha Bbl,Chemnitz,Gottingen,173.5,1.91,0.6319,Over
25,11:15,Grécia Liga De Basquete,Kolossos Rhodes,Olympiakos,156.5,1.91,0.6063,Over
26,11:30,França Lnb,Strasbourg,Monaco,163.5,1.91,0.6403,Over
28,12:00,Alemanha Bbl,Alba Berlin,Vechta,174.5,1.91,0.6178,Over
29,12:00,Espanha Acb,Valencia,Manresa,168.5,1.91,0.6362,Over
30,12:00,Itália Liga A,Treviso,Venezia,160.5,1.91,0.6059,Over


### Over v3 (Blend)

In [12]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['HA_Line'] < -8.5) | (prev['HA_Line'] > -5.5))]
    prev = prev[((prev['CV_ML'] >= 0) & (prev['CV_ML'] < 0.65)) | ((prev['CV_ML'] >= 0.77) & (prev['CV_ML'] < 0.87)) | ((prev['CV_ML'] >= 0.95) & (prev['CV_ML'] < 1.5))]


    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (BLEND) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (BLEND) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,Bet
15,08:00,Espanha Acb,Tenerife,Real Madrid,166.5,1.91,Over
21,09:30,França Lnb,Lyon-Villeurbanne,Roanne,164.5,1.91,Over
25,11:15,Grécia Liga De Basquete,Kolossos Rhodes,Olympiakos,156.5,1.91,Over
26,11:30,França Lnb,Strasbourg,Monaco,163.5,1.91,Over
30,12:00,Itália Liga A,Treviso,Venezia,160.5,1.91,Over
37,14:15,Grécia Liga De Basquete,Aris,AS Karditsas,148.5,1.91,Over
38,14:30,Itália Liga A,Pistoia,Brescia,158.5,1.91,Over
39,15:00,Itália Liga A,Virtus Bologna,Sassari,162.5,1.91,Over
